# TP3 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Enunciado do Problema

O seguinte sistema dinâmico denota 4 inversores ($A, B, C, D$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.

![](https://paper-attachments.dropboxusercontent.com/s_9896551CC5FAD2B2EB6E4EBC08522545FA66314D29FE6A5BE8E593259F8E8A37_1669554332522_inversores.png)

 1. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
 2. Cada inversor é regido pelas seguintes transformações:
 $$
   \mathbf{invert}\mathtt(in,out) \\
   x \gets \mathsf{read}(\mathtt{in}) \\
   s \gets \neg x\;\;\|\;\; s\gets s\oplus x \\
   \mathsf{write}(\mathtt{out},s) \\
 $$
 3. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicialização do sistema.
 4. O estado do sistema é um tuplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
 5. O sistema termina em ERRO quando o estado do sistema for $(0,0,0,0)$.

Prentede-se o seguinte:
 1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $n$ estados.
 2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.


## Análise

O problema exposto tem certas considerações a ter em conta.

Em primeiro lugar, considera-se o $in$ de cada um dos inversores igual ao $out$ do estado anterior, ou seja, o estado anterior com uma rotação cíclica das variáveis, exceto no caso do estado inicial. A ordem da rotação é a descrita nos canais, $a \rightarrow b \rightarrow d \rightarrow c$

No caso do estado inicial, decidiu-se gerar o $in$ aleatoriamente tal como se gera o $s$ para cada inversor, isto é, o estado inicial.

Em segundo lugar, como cada inversor funciona em paralelo com cada outro inversor, as variáveis de cada estado vão ser apenas cada um dos elementos de cada tuplo de $\{0,1\}^4$, não existindo variável que seja considerada o contador do programa. 

Em terceiro lugar, deve-se definir aleatoriamente cada um dos comportamentos dos inversores, que será mantido durante a execução inteira.

### Definição do SFOTS

Tal como pretendido, o sistema dinâmico será modelado através um SFOTS, nomeadamente um tuplo:
$$ \Sigma \; \equiv \; <\mathcal{T},X,next,I,T,E> $$

Onde se verifica o seguinte, para representar o sistema em específico:
 1. $\mathcal{T}$ representa um SMT apropriada, que pertence à FOL, que vamos representar no nosso Solver;
 2. $X$ é o conjunto das variáveis base do problema;
 3. $next$ é um operador que gera "clones" das variáves em $X$;
 4. $I$ é um predicado unário que determina quais os estados iniciais;
 5. $T$ é um predicado binário que determina as transições entre dois estados;
 6. $E$ é um predicado unário que determina os estados de erro.

$X$ neste caso é constituído por uma variável binária para cada um dos estados dos inversores, $\{s_a,s_b,s_c,s_d\}$. Dependendo da transição ciclica, um dos estados de um dos inversores será considerado o valor de entrada de outro. Ou seja, $in_a = s_c$, $in_b = s_a$, $in_c = s_d$ e $in_d = s_b$.

Considerando o estado inicial, não existem condições que limitam as variáveis, já que é possível o primeiro estado ser um estado de erro.

O predicado de transição depende da definição aleatória no início da execução. No seu geral, é definido por uma conjunção entre todos os inversores, cada um que pode ser um de dois predicados. Por exemplo, para o inversor $A$:

$$ T_a \; \equiv \; s_a' = \neg\; s_c \quad \text{ou} \quad T_a \; \equiv \; s_a' = s_a \oplus s_c$$

O estado de erro, tal como definido no enunciado, ocorre quando cada um dos $s$ é igual a $0$.

$$ E \quad \equiv \quad s_a = 0 \wedge s_b = 0 \wedge s_c = 0 \wedge s_d = 0$$

## Implementação

Para a resolução do problema em questão, decidiu-se usar o módulo `pysmt.shortcuts`, com as funcionalidades possíveis para a utilização de um SMT Solver. Importam-se também os tipos deste Solver, a partir do módulo `pysmt.typing`. Para modelar este problema, usar-se-ão variáveis binárias, então importamos o tipo `BOOL`. Também importamos a função `binomial` do módulo `numpy.random` para poder gerar lançamentos de moedas aleatórios.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL

from numpy.random import binomial
import itertools 

Para gerar os estados, recebem-se as variáveis do problema e, para cada uma, cria-se uma variável binária. Também se marca a variável com uma letra `s` que representa o conjunto de variáveis, nomeadamente $X$ ou $Y$, tal como um número `i` que representa a i-ésima cópia da variável.

In [2]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),BOOL)
    return state

Para se gerar os estados iniciais e as transições, é preciso definir aleatoriamente tanto as variáveis do estado inicial tal como qual das transições será utilizada por cada inversor.

In [3]:
def genInitialRandom():
    s = {}
    s['a']= binomial(1,0.5) == 1
    s['b'] = binomial(1,0.5) == 1
    s['c'] = binomial(1,0.5) == 1
    s['d'] = binomial(1,0.5) == 1
    print("Estados inciais:",s['a'],s['b'],s['c'],s['d'])
    return s

def genTransRandom():
    t = {}
    t['a'] = binomial(1,0.5) == 1
    t['b'] = binomial(1,0.5) == 1
    t['c'] = binomial(1,0.5) == 1
    t['d'] = binomial(1,0.5) == 1
    print("Transição Determinada:", "Neg" if t['a'] else "XOR", "Neg" if t['b'] else "XOR", "Neg" if t['c'] else "XOR", "Neg" if t['d'] else "XOR")
    return t

As variáveis do problema são os estados internos de cada inversor, que coincidem com o seu $out$. Dessa forma, não é necessário definir o $in$ dos inversores.


In [4]:
vars = ['s_a','s_b','s_c','s_d'] #in_a = s_c, in_b = s_a, in_c = s_d, in_d = s_b

A função inicial depende dos valores aleatórios gerados, apenas limitando cada uma das variáveis a esse valor, através de uma equivalência.
`init1` gera os valores aleatórios dentro da função enquanto que `init_notrandom` necessita de receber os valores gerados como parâmetro. Desta forma, cada função que as use será diferente na escrita de acordo com a função pretendida.

In [5]:
def init1(state):
    s = genInitialRandom()
    return And(Iff(state['s_a'], Bool(s['a'])), Iff(state['s_b'], Bool(s['b'])), 
               Iff(state['s_c'], Bool(s['c'])), Iff(state['s_d'], Bool(s['d'])))

def init_notrandom(state,a,b,c,d):
    return And(Iff(state['s_a'], Bool(a)), Iff(state['s_b'], Bool(b)), 
               Iff(state['s_c'], Bool(c)), Iff(state['s_d'], Bool(d)))

A função `error1` é semelhante às iniciais, mas limita exatamente o estado $(0,0,0,0)$ como o estado de erro.

In [6]:
def error1(state):
    return And(Iff(state['s_a'], Bool(False)), Iff(state['s_b'], Bool(False)), 
               Iff(state['s_c'], Bool(False)), Iff(state['s_d'], Bool(False)))

A função de transição `trans1` depende também dos valores gerados, e que recebe como parâmetro. Para cada um desses valores, escolhe-se uma das transições que será sempre a mesma (já que os valores são gerados apenas uma vez). Neste caso, `True` representa a transição $s \gets \neg x\;\;$ e `False` a transição $\;\; s\gets s\oplus x $.

In [7]:
def trans1(curr, prox, a, b, c, d):
    t_a = Iff(Not(curr['s_c']), prox['s_a']) if a else Iff(Xor(curr['s_c'], curr['s_a']), prox['s_a'])
    t_b = Iff(Not(curr['s_a']), prox['s_b']) if b else Iff(Xor(curr['s_a'], curr['s_b']), prox['s_b'])
    t_c = Iff(Not(curr['s_d']), prox['s_c']) if c else Iff(Xor(curr['s_d'], curr['s_c']), prox['s_c'])
    t_d = Iff(Not(curr['s_b']), prox['s_d']) if d else Iff(Xor(curr['s_b'], curr['s_d']), prox['s_d'])
    #in_cond = And(Iff(curr['in_a'], prox['in_b']), Iff(curr['in_b'], prox['in_d']), Iff(curr['in_c'], prox['in_a']), Iff(curr['in_d'], prox['in_c']))
    return And(t_a, t_b, t_c, t_d)

A função `genTrace` recebe todos estes argumentos mais o número `n`, tamanho máximo do traço, e devolve um traço execução sem qualquer consideração pelo estado de erro.

In [85]:
def genTrace(vars,init,trans,n,s,t):
    with Solver(name="z3") as solver:
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0],s['a'],s['b'],s['c'],s['d'])
        Tks = [ trans(X[i],X[i+1],t['a'],t['b'],t['c'],t['d']) for i in range(n) ]
        
        if solver.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',solver.get_value(X[i][v]))

In [104]:
t = genTransRandom()
s = genInitialRandom()
genTrace(vars, init_notrandom, trans1, 5,s,t)

Transição Determinada: Neg Neg Neg Neg
Estados inciais: False True True True
Estado: 0
           s_a = False
           s_b = True
           s_c = True
           s_d = True
Estado: 1
           s_a = False
           s_b = True
           s_c = False
           s_d = False
Estado: 2
           s_a = True
           s_b = True
           s_c = True
           s_d = False
Estado: 3
           s_a = False
           s_b = False
           s_c = True
           s_d = False
Estado: 4
           s_a = False
           s_b = True
           s_c = True
           s_d = True


## Verificação

### Verificação de Segurança

#### Indução

In [10]:
def induction_always(vars,gen_state,init,trans,inv,s,t):
    with Solver(name="z3") as solver:
        state0 = gen_state(vars, 'X', 0)
        state1 = gen_state(vars, 'X', 1)
        solver.push()
        solver.add_assertion(And(init(state0,s['a'],s['b'],s['c'],s['d']), Not(inv(state0))))
        if solver.solve():
            print("A propriedade não é válida no estado inicial.")
            for v in state0:
                print(v,"=",solverget_value(state0[v]))
            return
        solver.pop()
        
        solver.add_assertion(And(inv(state0), trans(state0,state1,t['a'],t['a'],t['a'],t['a']), Not(inv(state1))))
        if solver.solve():
            print("O passo indutivo não preserva a propriedade.")
            for v in state0:
                print(v,"=",solver.get_value(state0[v]))
            return
        print("A propriedade é valida.")

Este invariante não é provável por indução, já que não nos impede de chegar ao estado $(0,0,0,0)$ quando este não é acessível.

In [11]:
def inv1(state):
    return Not(error1(state))

In [12]:
s = genInitialRandom()
t = genTransRandom()
induction_always(vars, genState, init_notrandom, trans1, inv1,s,t)

Estados inciais: False False True False
Transição Determinada: XOR XOR XOR XOR
O passo indutivo não preserva a propriedade.
s_a = True
s_b = True
s_c = True
s_d = True


Sem `k-indução`, ou outro método, não se consegue verificar a acessibilidade dos estados inseguros, considerando o sistema inseguro mesmo que não o seja.

#### K-Indução

In [70]:
def kinduction_always(vars, gen_state,init,trans,inv,k,s,t):
    with Solver(name="z3") as solver:
        traco = [gen_state(vars, 'X', i) for i in range(k+1) ]
        solver.push()
        solver.add_assertion( init(traco[0], s['a'], s['b'], s['c'], s['d'])  )
        for e in range(k-1):
            solver.add_assertion( trans (traco[e], traco[e+1], t['a'], t['b'], t['c'], t['d'] ) )
        solver.add_assertion( Or([Not(inv(traco[i])) for i in range(k) ]) )

        if solver.solve():
            print("A propriedade não é valida para os k estados iniciais")
            i = 0
            for trace in traco:
                print("Estado:",i)
                i+=1
                for v in trace:
                    print("          ",v,'=',solver.get_value(trace[v]))
            return
        solver.pop()

        solver.push()
        for e in range(k):
            solver.add_assertion( trans (traco[e], traco[e+1], t['a'], t['b'], t['c'], t['d'] ) )
            solver.add_assertion( inv(traco[e]) )

        solver.add_assertion( Not(inv(traco[k])) )

        if solver.solve():
            print(f"O passo indutivo %d não preserva a propriedade" % k)
            i = 0
            for trace in traco:
                print("Estado:",i)
                i += 1
                for v in trace:
                    print("          ",v,'=',solver.get_value(trace[v]))
            return
        solver.pop()

        print("A propriedade é valida para todos os estados acessíveis ")

### Exemplos

Como ter a certeza? Se ele disser que é válida, tens a certeza que é válido. Se ele falhar para os casos iniciais

In [71]:
t = {'a': True, 'b': True, 'c': True, 'd': True}
s = {'a': False, 'b': True, 'c': True, 'd': True}
kinduction_always(vars, genState,init_notrandom,trans1,inv1,2,s,t)

A propriedade é valida para todos os estados acessíveis 


In [110]:
t = {'a': True, 'b': True, 'c': True, 'd': False}
s = {'a': True, 'b': True, 'c': True, 'd': False}
kinduction_always(vars, genState,init_notrandom,trans1,inv1,15,s,t)

A propriedade não é valida para os k estados iniciais
Estado: 0
           s_a = True
           s_b = True
           s_c = True
           s_d = False
Estado: 1
           s_a = False
           s_b = False
           s_c = True
           s_d = True
Estado: 2
           s_a = False
           s_b = True
           s_c = False
           s_d = True
Estado: 3
           s_a = True
           s_b = True
           s_c = False
           s_d = False
Estado: 4
           s_a = True
           s_b = False
           s_c = True
           s_d = True
Estado: 5
           s_a = False
           s_b = False
           s_c = False
           s_d = True
Estado: 6
           s_a = True
           s_b = True
           s_c = False
           s_d = True
Estado: 7
           s_a = True
           s_b = False
           s_c = False
           s_d = False
Estado: 8
           s_a = True
           s_b = False
           s_c = True
           s_d = False
Estado: 9
           s_a = False
           s_b

In [114]:
t = genTransRandom()
s = genInitialRandom()
kinduction_always(vars, genState,init_notrandom,trans1,inv1,10,s,t)

Transição Determinada: XOR XOR Neg XOR
Estados inciais: False False True False
A propriedade não é valida para os k estados iniciais
Estado: 0
           s_a = False
           s_b = False
           s_c = True
           s_d = False
Estado: 1
           s_a = True
           s_b = False
           s_c = True
           s_d = False
Estado: 2
           s_a = False
           s_b = True
           s_c = True
           s_d = False
Estado: 3
           s_a = True
           s_b = True
           s_c = True
           s_d = True
Estado: 4
           s_a = False
           s_b = False
           s_c = False
           s_d = False
Estado: 5
           s_a = False
           s_b = False
           s_c = True
           s_d = False
Estado: 6
           s_a = True
           s_b = False
           s_c = True
           s_d = False
Estado: 7
           s_a = False
           s_b = True
           s_c = True
           s_d = False
Estado: 8
           s_a = True
           s_b = True
           

### Algoritmo de "model-checking" usando interpolantes e invariantes

In [15]:
def invert(trans,a,b,c,d):
    return (lambda curr,prox: trans(prox, curr, a, b, c, d))

In [16]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Iff(state1[x],state2[x]) for x in state1])

In [120]:
def model_checking(vars,init,trans,error,N,M,s,t):
    with Solver(name="msat") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        #falta testar para n = 0 e m = 0
        
        for (n,m) in order:
            I = init(X[0],s['a'],s['b'],s['c'],s['d'])
            E = error(Y[0])
            Tn = And([trans(X[i], X[i+1],t['a'],t['b'],t['c'],t['d']) for i in range(n)])
            Bm = And([invert(trans,t['a'],t['b'],t['c'],t['d'])(Y[j], Y[j+1]) for j in range(m)])
            Rn = And(I, Tn)
            #print(Rn)
            Um = And(E, Bm)
            Vnm = And(Rn, Um, same(X[n], Y[m]))
            
            #1º Passo
            if solver.solve([Vnm]):
                print("Unsafe!")
                return
            
            #2º Passo
            C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
            if n == 1 and m == 1:
                print("C:", C)
            if C is None:
                print("Interpolante None!")
                continue
            #3ª Passo
            C0 = rename(C, X[0])
            C1 = rename(C, X[1])
            T = trans(X[0], X[1],t['a'],t['b'],t['c'],t['d'])
            if not solver.solve([C0, T, Not(C1)]):
                print("Safe!")
                return
            
            #4º Passo - gerar o S
            S = rename(C, X[n])
            while True:
                A = And(S, trans(X[n], Y[m],t['a'],t['b'],t['c'],t['d']))
                if solver.solve([A, Um]):
                    """print("Xn:")
                    for v in X[n]:
                        print("          ",v,'=',solver.get_value(X[n][v]))
                    for i in range(m+1):
                        print(f"Y%d:" % i)
                        for v in Y[i]:
                            print("          ",v,'=',solver.get_value(Y[i][v]))"""
                    print("Não se encontrou o Majorante.")
                    break
                Cnew = binary_interpolant(A, Um)
                if Cnew is None:
                    print("Interpolante None!")
                    break
                Cn = rename(Cnew, X[n])
                if solver.solve([Cn, Not(S)]):
                    S = Or(S, Cn)
                else:
                    print("Safe!")
                    break
                
            
        print("unknown")                        

Os seguintes dados iniciais geram um sistema seguro, mas a geração dos interpolantes apenas gera condições que não são limitantes o suficiente, incapacitando a geração de um majorante que não intersete com $U_m$. Tal verifica-se nos casos que supostamente são seguros, como quando 

O interpolante sendo só $\neg X1_c$ faz com que seja possível considerar $X_n$ como o estado $(0,0,0,0)$, que é claramente inseguro.
Para $X1 = (1,0,0,0)$ e $Y1 = (1,1,1,1)$, com todas as transições definidas para `neg`, um interpolante não será forte o suficiente para excluir a possibilidade de $Xn$ ser $(0,0,0,0)$.

In [121]:
t = {'a':True,'b':True,'c':True,'d':True}
s = {'a':True,'b':True,'c':True,'d':False}
genTrace(vars,init_notrandom,trans1,2,s,t)
kinduction_always(vars, genState,init_notrandom,trans1,inv1,2,s,t)
model_checking(vars, init_notrandom, trans1, error1, 5, 5,s,t)

Estado: 0
           s_a = True
           s_b = True
           s_c = True
           s_d = False
Estado: 1
           s_a = False
           s_b = False
           s_c = True
           s_d = False
A propriedade é valida para todos os estados acessíveis 
C: (! s_a!Y1)
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se encontrou o Majorante.
Não se en

In [129]:
t = genTransRandom()
s = genInitialRandom()
model_checking(vars, init_notrandom, trans1, error1, 10, 10,s,t)

Transição Determinada: XOR Neg XOR Neg
Estados inciais: False False True False
Unsafe!


É possível provar a impossibilidade de calcular um interpolante que seja um invariante pretendido para o caso que todas as operações sejam negações.

Um sistema destes que seja seguro entra necessariamente num ciclo, já que não há caso não inseguro de paragem. Possivelmente, isso sugere que não seja possível definir interpolantes fortes o suficiente. Pode ser impossível calcular interpolantes que sejam invariantes para os casos seguros.

## Exemplos